In [ ]:

# Import library

# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
import pandas as pd

from amiral import instructment, utils, parameter, gradient, minimisation, array
from scipy.optimize import minimize 
%matplotlib inline

import tools

from mpl_toolkits.axes_grid1 import make_axes_locatable



In [ ]:
# Function for getting snr
def get_snr (array, noise):
    
    mean = np.mean(array)
    sig2 = np.std(noise)
    
    snr = mean / sig2
    
    return snr

def create_psfao19_otf (otf_tel, guess, aosys_cls): 
    
    # Use PSFAO19 model to create a PSF
    psd_ao = aosys_cls.psd_residual_ao (guess = guess)
    psd_halo = aosys_cls.psd_residual_halo(r0 = guess[0])
    
    psd = psd_ao + psd_halo

    otf_atmo = aosys_cls.otf_atmo(psd)
    otf_total = otf_atmo*otf_tel
    
    
    return otf_atmo,otf_total

def strehl (psf, psf_tel):
    
    return np.max(psf)/np.max(psf_tel)

In [ ]:
# PATH

nb = 9
case = 'VESTA'

source_path = "/Users/alau/Data/amiral_fits/VESTA/SNR/2021apr23/case_" +str(nb) + "/"
est_csv_path = "/Users/alau/Data/amiral_fits/VESTA/SNR/2021apr23/case_"+str(nb) + "/"

# source_csv_fname =  case + "_SNR_data_"+str(nb)+".csv"
# est_csv_fname = case + "_SNR_"+str(nb)+"_r0_sig2_mu_rho0_299.csv"

source_csv_fname =  case + "_SNR_data_"+str(nb)+".csv"
est_csv_fname = case + "_SNR"+"_r0_sig2_mu_rho0_"+str(nb)+"_299.csv"

Get the object

In [ ]:
# high_snr_source_csv = pd.read_csv(high_snr_source_csv_path+high_snr_source_csv_fname)
# high_snr_est_csv = pd.read_csv(high_snr_est_csv_path+high_snr_est_csv_fname)

# low_snr_source_csv = pd.read_csv(low_snr_source_csv_path+low_snr_source_csv_fname)
# low_snr_est_csv = pd.read_csv(low_snr_est_csv_path+low_snr_est_csv_fname)


# true_r0 = high_snr_source_csv['r0']
# true_sig2 = high_snr_source_csv['amplitude']
# true_flux = high_snr_source_csv['flux']
# snr = high_snr_source_csv['snr']

# est_r0 = high_snr_est_csv['r0']
# est_sig2 = high_snr_est_csv['sig2']

# source_frame  = [low_snr_source_csv,high_snr_source_csv]
# results_frame = [low_snr_est_csv,high_snr_est_csv]

# result = pd.concat(results_frame).reset_index(drop=True)

# source = pd.concat(source_frame).reset_index(drop=True)

source = pd.read_csv(source_path+source_csv_fname)
result = pd.read_csv(est_csv_path+est_csv_fname)

true_r0 = source['r0']
true_sig2 = source['amplitude']
true_flux = source['flux']
snr = source['snr']

est_r0 = result['r0']
est_sig2 = result['sig2']

We need a PSF

In [ ]:
aosys_dict = {
    'diameter': 8 , 
    'occ_ratio': 0.14 , 
    'no_acutuator' : 20, 
    'wavelength': 500,  # 165
    'dimension': 256,
    'resolution_rad' : 2.083e-8
}

amiral_dict = {
    "r0": true_r0[0],                 
    "background": 0.0,      
    "amplitude": true_sig2[0],   
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

psf_keys, psf_guess = utils.dict2array(amiral_dict)



aosys_cls = instructment.aoSystem( 
        diameter = aosys_dict['diameter'], 
        occ_ratio = aosys_dict['occ_ratio'], 
        no_acutuator= aosys_dict['no_acutuator'], 
        wavelength = aosys_dict['wavelength']*1e-9, 
        resolution_rad = aosys_dict['resolution_rad'], 
        dimension=aosys_dict['dimension'])  


pupil = aosys_cls.get_pupil_plane()
otf_tel = aosys_cls.pupil_to_otf_tel(pupil)
psf_tel = np.fft.fftshift(np.real(utils.ifft2D(otf_tel)))

psf_guess[0] = true_r0[0]
psf_guess[2] = true_sig2[0]

otf_atmo, otf_total = create_psfao19_otf(otf_tel, psf_guess, aosys_cls)
psf = np.fft.fftshift(np.real(utils.ifft2D(otf_total)))

sr = strehl(psf, psf_tel)


print(sr)

In [ ]:
est_sr_list = []
sr_list = []

for i in range (len(true_r0)):    
    psf_guess[0] = est_r0[i]
    psf_guess[2] = est_sig2[i]
    _otf_atmo, _otf_total = create_psfao19_otf(otf_tel, psf_guess, aosys_cls)
    _psf = np.real(np.fft.ifft2(np.fft.fftshift(_otf_total)))
    _sr = strehl(_psf, psf_tel)
        
    est_sr_list.append(_sr)
    sr_list.append(sr)

In [ ]:
est_sr_list = np.asarray(est_sr_list)

sr_list = np.asarray(sr_list)

In [ ]:
rcParams['figure.figsize'] = 21,18
fig, ax = plt.subplots(3,1)
fig.tight_layout(pad = 8.0)


ax[0].set_title('r0 = %f sig2 = %f' %(true_r0[0],true_sig2[0]), fontsize = 18)
ax[0].set_ylabel(r'$\mathrm{r_0[cm] error}$', fontsize = 16)
ax[0].set_xlabel(r'$\mathrm{Flux}$', fontsize = 16)
ax[0].set_xscale("log")


ax[1].axhline(y=0, color = 'r', ls = '--')
ax[1].set_xlabel(r'$\mathrm{Flux}$', fontsize = 16)
ax[1].set_xscale("log")
ax[1].set_ylabel(r'$\mathrm{\sigma^2[rad^2] error}$', fontsize = 16)


ax[2].set_ylabel(r'$\mathrm{S_R error(\%)error}$', fontsize = 16)
ax[2].set_xlabel(r'$\mathrm{Flux}$', fontsize = 16)
ax[2].axhline(y=0, color = 'r', ls = '--')
ax[2].set_xscale("log")


ax[0].axhline(y=0, color = 'r', ls = '--')

length = np.size(true_flux)

pack = length//10
n_frame = 10

for i in range (pack):
    for j in range (n_frame):

        ax[0].scatter(true_flux[i*n_frame+j],100*(est_r0-true_r0)[i*n_frame+j])
        ax[1].scatter(true_flux[i*n_frame+j],(est_sig2- true_sig2)[i*n_frame+j])
        ax[2].scatter(true_flux[i*n_frame+j],100*(est_sr_list - sr_list)[i*n_frame+j])

In [ ]:
rcParams['figure.figsize'] = 21,18
fig, ax = plt.subplots(3,1)
fig.tight_layout(pad = 8.0)


ax[0].set_title('r0 = %f sig2 = %f' %(true_r0[0],true_sig2[0]), fontsize = 16)
ax0 = ax[0].twiny()
ax0.set_xlabel(r'$\mathrm{Flux}$', fontsize = 14)
ax0.set_xscale("log")

ax[0].set_ylabel(r'$\mathrm{r_0[cm] error}$', fontsize = 14)
# ax[0].set_xlabel(r'$\mathrm{Frame number}$', fontsize = 14)

ax1 = ax[1].twiny()
ax1.set_xlabel(r'$\mathrm{Flux}$', fontsize = 14)
ax1.set_xscale("log")

ax[1].axhline(y=0, color = 'r', ls = '--')
ax[1].set_xlabel(r'$\mathrm{flux}$', fontsize = 14)

ax[1].set_ylabel(r'$\mathrm{\sigma^2[rad^2] error}$', fontsize = 14)

ax2 = ax[2].twiny()
ax2.set_xlabel(r'$\mathrm{Flux}$', fontsize = 14)
ax2.set_xscale("log")

ax[2].set_ylabel(r'$\mathrm{S_R error(\%)error}$', fontsize = 14)
# ax[2].set_xlabel(r'$\mathrm{Frame number}$', fontsize = 14)
ax[2].axhline(y=0, color = 'r', ls = '--')

ax[0].fill_betweenx(est_r0-true_r0,1)
ax[0].axhline(y=0, color = 'r', ls = '--')



length = np.size(true_flux)

pack = length//10
n_frame = 10

for i in range (pack):
    for j in range (n_frame):
            ax0.scatter(true_flux[i*n_frame+j],100*(est_r0-true_r0)[i*n_frame+j])
#             ax[0].scatter(i*n_frame+j,100*(est_r0-true_r0)[i*n_frame+j])

#             ax[1].scatter(i*n_frame+j,(est_sig2- true_sig2)[i*n_frame+j])
            ax1.scatter(true_flux[i*n_frame+j],(est_sig2- true_sig2)[i*n_frame+j])
        
            ax2.scatter(true_flux[i*n_frame+j],100*(est_sr_list - sr_list)[i*n_frame+j])
            
#             ax[2].scatter(i*n_frame+j,100*(est_sr_list - sr_list)[i*n_frame+j])    

In [ ]:

# fig.savefig(case+"_data_"+str(nb)+".pdf", dpi = 300)

In [ ]:
# rcParams['figure.figsize'] = 16,9

# fig, ax = plt.subplots(3,1)

# ax[0].set_xlim(np.min(true_flux), np.max(true_flux))

# ax[0].scatter(true_flux,100*(est_r0 -true_r0), s = 10)
# ax[0].set_ylabel(r'$\mathrm{r_0[cm] error}$', fontsize = 14)
# ax[1].scatter(true_flux,100*(est_sr_list- sr_list),s = 10)
# ax[1].set_ylabel(r'$\mathrm{S_R error(\%)error}$', fontsize = 14)
# ax[2].scatter(true_flux,est_sig2- true_sig2,s = 10)
# ax[2].set_ylabel(r'$\mathrm{\sigma^2[rad^2] error}$', fontsize = 14)


In [ ]:
# rcParams['figure.figsize'] = 21,18

# fig, ax = plt.subplots(3,2,constrained_layout=True)
# # plt.subplots_adjust(hspace = .001)


# ax[0,0].set_title('r0 = %f sig2 = %f' %(true_r0[0],true_sig2[0]))
# ax[0,0].set_xlim(np.min(true_flux), np.max(true_flux))
# ax[0,0].set_xscale('log')
# ax[0,0].set_ylim(-2, 2)
# ax[0,0].scatter(true_flux,100*(est_r0 -true_r0))
# ax[0,0].set_ylabel(r'$\mathrm{r_0[cm] error}$', fontsize = 14)
# ax[0,0].set_xlabel(r'$\mathrm{flux}$', fontsize = 14)
# ax[0,0].fill_betweenx(est_r0-true_r0,1)
# ax[0,0].axhline(y=0, color = 'r', ls = '--')



# ax[1,0].set_xlim(np.min(true_flux), np.max(true_flux))
# ax[1,0].scatter(true_flux,est_sig2- true_sig2)
# ax[1,0].set_xlabel(r'$\mathrm{flux}$', fontsize = 14)
# ax[1,0].set_ylabel(r'$\mathrm{\sigma^2[rad^2] error}$', fontsize = 14)
# ax[1,0].set_ylim(-2, 2)
# ax[1,0].set_xscale('log')
# ax[1,0].axhline(y=0, color = 'r', ls = '--')



# ax[2,0].axhline(y=0, color = 'r', ls = '--')
# ax[2,0].scatter(true_flux,100*(est_sr_list - sr_list))
# ax[2,0].set_xlim(np.min(true_flux), np.max(true_flux))
# ax[2,0].set_xscale('log')
# ax[2,0].set_ylabel(r'$\mathrm{S_R error(\%)error}$', fontsize = 14)
# ax[2,0].set_ylim(-6, 6)

# ax[0,1].set_title('r0 = %f sig2 = %f' %(true_r0[0],true_sig2[0]))
# ax[0,1].set_xlim(np.min(true_flux), np.max(true_flux))
# ax[0,1].set_xscale('log')
# ax[0,1].scatter(true_flux,100*(est_r0-true_r0))
# ax[0,1].set_ylabel(r'$\mathrm{r_0[cm] error}$', fontsize = 14)
# ax[0,1].set_xlabel(r'$\mathrm{flux}$', fontsize = 14)
# ax[0,1].fill_betweenx(est_r0-true_r0,1)
# ax[0,1].axhline(y=0, color = 'r', ls = '--')



# ax[1,1].set_xlim(np.min(true_flux), np.max(true_flux))
# ax[1,1].scatter(true_flux,est_sig2- true_sig2)
# ax[1,1].set_xlabel(r'$\mathrm{flux}$', fontsize = 14)
# ax[1,1].set_ylabel(r'$\mathrm{\sigma^2[rad^2] error}$', fontsize = 14)
# ax[1,1].set_xscale('log')
# ax[1,1].axhline(y=0, color = 'r', ls = '--')



# ax[2,1].axhline(y=0, color = 'r', ls = '--')
# ax[2,1].set_xlim(np.min(true_flux), np.max(true_flux))
# ax[2,1].scatter(true_flux,100*(est_sr_list - sr_list))
# ax[2,1].set_xscale('log')
# ax[2,1].set_ylabel(r'$\mathrm{S_R error(\%)error}$', fontsize = 14)
# # ax[2].set_ylim(-6, 6)

# # fig.savefig("VESTA_SNR_data_"+str(nb)+".pdf", dpi= 300)



In [ ]:
# rcParams['figure.figsize'] = 21,18

# fig, ax = plt.subplots(3,1)

# ax[0].set_title('r0 = %f sig2 = %f' %(true_r0[0],true_sig2[0]))
# ax[0].set_xlim(np.min(true_flux), 2*np.max(true_flux))
# ax[0].set_xscale('log')
# # ax[0].set_ylim(-2, 2)
# ax[0].scatter(true_flux,100*(est_r0 -true_r0))
# ax[0].set_ylabel(r'$\mathrm{r_0[cm] error}$', fontsize = 14)
# ax[0].set_xlabel(r'$\mathrm{flux}$', fontsize = 14)
# ax[0].fill_betweenx(est_r0-true_r0,1)
# ax[0].axhline(y=0, color = 'r', ls = '--')



# ax[1].set_xlim(np.min(true_flux), 2*np.max(true_flux))
# ax[1].scatter(true_flux,est_sig2- true_sig2)
# ax[1].set_xlabel(r'$\mathrm{flux}$', fontsize = 14)
# ax[1].set_ylabel(r'$\mathrm{\sigma^2[rad^2] error}$', fontsize = 14)
# # ax[1].set_ylim(-2, 2)
# ax[1].set_xscale('log')
# ax[1].axhline(y=0, color = 'r', ls = '--')



# ax[2].axhline(y=0, color = 'r', ls = '--')
# ax[2].scatter(true_flux,100*(est_sr_list - sr_list))
# ax[2].set_xlim(np.min(true_flux), 2*np.max(true_flux))
# ax[2].set_xscale('log')
# ax[2].set_ylabel(r'$\mathrm{S_R error(\%)error}$', fontsize = 14)
# ax[2].set_xlabel(r'$\mathrm{flux}$', fontsize = 14)
# # ax[2].set_ylim(-1, 1)

# # fig.savefig("VESTA_SNR_data_9.pdf", dpi= 300)


Check the noise

In [ ]:
# # img = obs_image
# source_csv_path = "/Users/alau/Data/amiral_fits/"+ case +"/SNR/sensitivity_analysis/2021apr06/case_"+str(nb)+"/"
# source_csv_path_1 = "/Users/alau/Data/amiral_fits/"+ case +"/SNR/sensitivity_analysis/2021apr12/case_"+str(nb)+"/low_snr/"


# # output = "2018-06-08T05_27_05.809_196.fits"

# ref = 0
# count = 199

# noise_output = "2018-06-08T05_27_05.809_noise_"+str(ref)+".fits"
# fits_data = "2018-06-08T05_27_05.809_"+str(count)+".fits"

# noise_list = np.linspace(ref,count,200)

# noise = []
# data = []
# residual = []

# for ind, noise_list in enumerate(noise_list):    
#     count = str(int(noise_list))
    
#     noise_output = "2018-06-08T05_27_05.809_noise_"+str(count)+".fits"
#     _noise_data = fits.open(source_csv_path_1+noise_output)
#     _noise_data = _noise_data[0].data
    
#     fits_data = "2018-06-08T05_27_05.809_"+str(count)+".fits"
#     _data = fits.open(source_csv_path_1+fits_data)
#     _data = _data[0].data
        
#     noise.append(_noise_data)
#     data.append(_data)
#     residual.append(np.sum(_data-_noise_data))

In [ ]:
# noise_list = np.linspace(0,199,200)
# for ind, noise_list in enumerate(noise_list):    
#     count = str(int(noise_list))

#     noise_output = "2018-06-08T05_27_05.809_noise_"+str(count)+".fits"
#     _noise_data = fits.open(source_csv_path+noise_output)
#     _noise_data = _noise_data[0].data

#     fits_data = "2018-06-08T05_27_05.809_"+str(count)+".fits"
#     _data = fits.open(source_csv_path+fits_data)
#     _data = _data[0].data

#     noise.append(_noise_data)
#     data.append(_data)
#     residual.append(np.sum(_data-_noise_data))

In [ ]:
# fig, ax = plt.subplots(5,1)
# rcParams['figure.figsize'] = 21,16
# fig.tight_layout(pad = 8.0)

# from cycler import cycler


# ax[0].set_prop_cycle(cycler('color', ['b', 'g', 'r', 'c', 'm', 'y', 'k','gray','lime','pink']) +
#                    cycler('lw', [1, 2, 3, 4,5,6,7,8,9,10]))

# ax[0].set_title('r0 = %f sig2 = %f' %(true_r0[0],true_sig2[0]))
# ax0 = ax[0].twiny()
# ax0.set_xlabel(r'$\mathrm{Flux}$', fontsize = 14)
# ax0.set_xscale("log")


# ax[1].set_prop_cycle(cycler('color', ['b', 'g', 'r', 'c', 'm', 'y', 'k','gray','lime','pink']) +
#                    cycler('lw', [1, 2, 3, 4,5,6,7,8,9,10]))
# ax1 = ax[1].twiny()
# ax1.set_xlabel(r'$\mathrm{Flux}$', fontsize = 14)
# ax1.set_xscale("log")

# ax[2].set_prop_cycle(cycler('color', ['b', 'g', 'r', 'c', 'm', 'y', 'k','gray','lime','pink']) +
#                    cycler('lw', [1, 2, 3, 4,5,6,7,8,9,10]))

# ax2 = ax[2].twiny()
# ax2.set_xlabel(r'$\mathrm{Flux}$', fontsize = 14)
# ax2.set_xscale("log")


# ax[3].set_prop_cycle(cycler('color', ['b', 'g', 'r', 'c', 'm', 'y', 'k','gray','lime','pink']) +
#                    cycler('lw', [1, 2, 3, 4,5,6,7,8,9,10]))

# ax3 = ax[3].twiny()
# ax3.set_xlabel(r'$\mathrm{Flux}$', fontsize = 14)
# ax3.set_xscale("log")


# ax[4].set_prop_cycle(cycler('color', ['b', 'g', 'r', 'c', 'm', 'y', 'k','gray','lime','pink']) +
#                    cycler('lw', [1, 2, 3, 4,5,6,7,8,9,10]))

# ax4 = ax[4].twiny()
# ax4.set_xlabel(r'$\mathrm{Flux}$', fontsize = 14)
# ax4.set_xscale("log")


# length = np.size(true_flux)

# pack = length//10
# n_frame = 10

# for i in range (pack):
#     for j in range (n_frame):
#         ax0.plot(true_flux[i*n_frame+j], 100*(est_r0-true_r0)[i*n_frame+j])
#         ax[0].scatter(i*n_frame+j,100*(est_r0-true_r0)[i*n_frame+j])


#         ax1.scatter(true_flux[i*n_frame+j],(est_sig2- true_sig2)[i*n_frame+j])
#         ax[1].scatter(i*n_frame+j,(est_sig2- true_sig2)[i*n_frame+j])

#         ax2.scatter(true_flux[i*n_frame+j],100*(est_sr_list - sr_list)[i*n_frame+j])              
#         ax[2].scatter(i*n_frame+j,100*(est_sr_list - sr_list)[i*n_frame+j])

#         ax3.scatter(true_flux[i*n_frame+j],100*(residual[i*n_frame+j]-true_flux[i*n_frame+j])/true_flux[i*n_frame+j])
#         ax[3].scatter(i*n_frame+j,100*(residual[i*n_frame+j]-true_flux[i*n_frame+j])/true_flux[i*n_frame+j])
        
        
#         ax4.scatter(true_flux[i*n_frame+j],np.sum(noise[i*n_frame+j]))
#         ax[4].scatter(i*n_frame+j,np.sum(noise[i*n_frame+j]))


# ax[0].set_ylabel(r'$\mathrm{r_0[cm] error}$', fontsize = 14)
# ax[0].set_xlabel(r'$\mathrm{Frame number}$', fontsize = 14)
# ax[0].fill_betweenx(est_r0-true_r0,1)
# ax[0].axhline(y=0, color = 'r', ls = '--')
# ax[0].set_ylim(-10, 10)
# ax[0].set_xlim(140, 400)

# ax[1].set_xlabel(r'$\mathrm{Frame Number}$', fontsize = 14)
# ax[1].set_ylabel(r'$\mathrm{\sigma^2[rad^2] error}$', fontsize = 14)
# ax[1].axhline(y=0, color = 'r', ls = '--')
# ax[1].set_xlim(140, 400)

# ax[2].set_ylabel(r'$\mathrm{S_R error(\%)error}$', fontsize = 14)
# ax[2].set_xlabel(r'$\mathrm{Frame Number}$', fontsize = 14)
# ax[2].axhline(y=0, color = 'r', ls = '--')
# ax[2].set_ylim(-20, 20)
# ax[2].set_xlim(140, 400)

# ax[3].set_ylabel(r'$\mathrm{Residual (\%)}$', fontsize = 14)
# ax[3].set_xlabel(r'$\mathrm{Frame Number}$', fontsize = 14)
# ax[3].axhline(y=0, color = 'r', ls = '--')
# ax[3].set_ylim(-1,1)
# ax[3].set_xlim(140, 400)

# ax[4].set_yscale('log')
# ax[4].set_ylabel(r'$\mathrm{Noise (e-)}$', fontsize = 14)
# ax[4].set_xlabel(r'$\mathrm{Frame Number}$', fontsize = 14)
# ax[4].set_xlim(140, 400)

# # fig.savefig(case+"_SNR_data_"+str(nb)+".pdf", dpi= 300)

In [ ]:
100*(est_r0-true_r0)

In [ ]:
true_flux